# 이전 대화를 기억하는 Chain 생성방법

In [9]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [10]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH05-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Memory


## 이전 대화내용을 기억하는 multi-turn Chain

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용
    ]
)


In [12]:
# llm 생성
import os
from langserve import RemoteRunnable

llm = ChatOpenAI(model_name="gpt-3.5-turbo")
# llm = RemoteRunnable(f"{os.getenv('LANGSERVE_ENDPOINT')}/llm")

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()

대화를 기록하는 체인 생성(`chain_with_history`)

In [13]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 파일을 설정
def get_chat_history(conversation_id):
    return SQLChatMessageHistory(
        table_name="history",
        session_id=conversation_id,
        connection="sqlite:///sqlite_runnable.db",
    )

In [14]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [15]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
    history_factory_config=config_fields,
)

첫 번째 질문 실행

In [16]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "나의 이름은 테디입니다."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"conversation_id": "abc123"}},
)

'안녕하세요, 테디님! 무엇을 도와드릴까요?'

이어서 질문 실행

In [17]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름이 뭐라고?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"conversation_id": "abc123"}},
)

'당신의 이름은 테디입니다.'

아래는 `session_id`가 다른 경우 새로운 세션이 생성되는 경우입니다.

In [18]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름이 뭐라고?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"conversation_id": "abc1234"}},
)

'죄송합니다, 저는 사용자의 이름을 알 수 없습니다. 어떤 다른 질문에 대해 도와드릴까요?'